# Лабораторная работа № 4. Нейронные сети

Постройте нейросетевой классификатор для набора данных [MNIST](https://www.kaggle.com/c/digit-recognizer).

## Задание

1. Постройте двуслойную нейронную сеть.
2. Постройте отчет по классификации  [classification_report](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) и матрицу ошибок [confusion_matrix](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py)
3. Добавьте скрытые слои и оцените как изменится качество классификации.